In [1]:
!pip install ucimlrepo

In [2]:
from ucimlrepo import fetch_ucirepo
import pandas as pd

import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from sklearn.model_selection import train_test_split


from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Embedding, Flatten, Add, Input, Activation
from tensorflow.keras.utils import plot_model, to_categorical

from sklearn.preprocessing import MinMaxScaler

# fetch dataset
covertype = fetch_ucirepo(id=31)

# data (as pandas dataframes)
X = covertype.data.features
Y = covertype.data.targets

In [3]:
# metadata
print(covertype.metadata)

# variable information
# print(covertype.variables)

{'uci_id': 31, 'name': 'Covertype', 'repository_url': 'https://archive.ics.uci.edu/dataset/31/covertype', 'data_url': 'https://archive.ics.uci.edu/static/public/31/data.csv', 'abstract': 'Classification of pixels into 7 forest cover types based on attributes such as elevation, aspect, slope, hillshade, soil-type, and more.', 'area': 'Biology', 'tasks': ['Classification'], 'characteristics': ['Multivariate'], 'num_instances': 581012, 'num_features': 54, 'feature_types': ['Categorical', 'Integer'], 'demographics': [], 'target_col': ['Cover_Type'], 'index_col': None, 'has_missing_values': 'no', 'missing_values_symbol': None, 'year_of_dataset_creation': 1998, 'last_updated': 'Sat Mar 16 2024', 'dataset_doi': '10.24432/C50K5N', 'creators': ['Jock Blackard'], 'intro_paper': None, 'additional_info': {'summary': 'Predicting forest cover type from cartographic variables only (no remotely sensed data).  The actual forest cover type for a given observation (30 x 30 meter cell) was determined from

In [4]:
# X.head()

In [5]:
# X.describe()

In [6]:
# X.info()

In [7]:
Y.head()

,Cover_Type
0,5
1,5
2,2
3,2
4,5


In [8]:
Y['Cover_Type'] = Y['Cover_Type'] - 1

<ipython-input-8-629182affda7>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Y['Cover_Type'] = Y['Cover_Type'] - 1


In [9]:
np.unique(Y)

array([0, 1, 2, 3, 4, 5, 6])

In [10]:
Y = to_categorical(Y, 7)

In [11]:
scaler = MinMaxScaler()

#normalize the values
X_scaled = pd.DataFrame(scaler.fit_transform(X), columns=X.columns)

In [12]:
X_scaled.head()

,Elevation,Aspect,Slope,Horizontal_Distance_To_Hydrology,Vertical_Distance_To_Hydrology,Horizontal_Distance_To_Roadways,Hillshade_9am,Hillshade_Noon,Hillshade_3pm,Horizontal_Distance_To_Fire_Points,...,Soil_Type34,Soil_Type35,Soil_Type36,Soil_Type37,Soil_Type38,Soil_Type39,Soil_Type40,Wilderness_Area2,Wilderness_Area3,Wilderness_Area4
0,0.368684,0.141667,0.045455,0.184681,0.223514,0.071659,0.870079,0.913386,0.582677,0.875366,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.365683,0.155556,0.030303,0.151754,0.215762,0.054798,0.866142,0.925197,0.594488,0.867838,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.472736,0.386111,0.136364,0.191840,0.307494,0.446817,0.921260,0.937008,0.531496,0.853339,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.463232,0.430556,0.272727,0.173228,0.375969,0.434172,0.937008,0.937008,0.480315,0.865886,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.368184,0.125000,0.030303,0.109520,0.222222,0.054939,0.866142,0.921260,0.590551,0.860449,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [13]:
X_train, X_validation, Y_train, Y_validation = train_test_split(X_scaled, Y, test_size=0.2, shuffle = True, random_state=42)

In [14]:
X_train
print(f'train size: {len(X_train)}.  ({len(X_train)/(len(X_train) + len(X_validation))})')
print(f'validation size: {len(X_validation)}.  ({len(X_validation)/(len(X_train) + len(X_validation))})')

train size: 464809.  (0.7999989673190915)
validation size: 116203.  (0.2000010326809085)


In [15]:
numberOfColumns = 54
inputs = Input(shape=(numberOfColumns,))

x = Dense(64, activation="relu")(inputs)  # First Dense layer
x = Dense(32, activation="relu")(x)       # Second Dense layer

# residual block
residual_input = x # Save input for the residual connection

# dense layers inside the residual block
xRes = Dense(16, activation="relu")(x)
xRes = Dense(16, activation="relu")(xRes)

# ensures residual input will match same dimensions as the output of our residual block
if residual_input.shape[-1] != xRes.shape[-1]:
    residual_input = Dense(16)(residual_input)

# # # linear projection to match sizes
# lin_projection = Dense(16)(residual_input)


# adding the residual input to the output
# xRes = Add()([xRes, lin_projection])
xRes = Add()([xRes, residual_input])

# skip connection
skip_connection = Dense(16)(inputs)

xSkip = Add()([xRes, skip_connection])  # Combine residual block output with skip connection


# final layers
x = Dense(16, activation="relu")(xSkip)
x = Dense(16, activation="relu")(x)

output = Dense(7, activation="softmax")(x)

In [16]:
model = Model(inputs=inputs, outputs=output)

In [17]:
# Compile model
model.compile(optimizer="adam", loss="CategoricalCrossentropy", metrics=["accuracy"]) #sparse cross entropy?

In [18]:
model.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)  │ (None, 54)             │              0 │ -                      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense (Dense)             │ (None, 64)             │          3,520 │ input_layer[0][0]      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_1 (Dense)           │ (None, 32)             │          2,080 │ dense[0][0]            │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_2 (Dense)           │ (None, 16)             │            528 │ dense_1[0][0]          │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_3 (Dense)           │ (None, 16)             │            272 │ dense_2[0][0]          │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_4 (Dense)           │ (None, 16)             │            528 │ dense_1[0][0]          │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ add (Add)                 │ (None, 16)             │              0 │ dense_3[0][0],         │
│                           │                        │                │ dense_4[0][0]          │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_5 (Dense)           │ (None, 16)             │            880 │ input_layer[0][0]      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ add_1 (Add)               │ (None, 16)             │              0 │ add[0][0],             │
│                           │                        │                │ dense_5[0][0]          │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_6 (Dense)           │ (None, 16)             │            272 │ add_1[0][0]            │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_7 (Dense)           │ (None, 16)             │            272 │ dense_6[0][0]          │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_8 (Dense)           │ (None, 7)              │            119 │ dense_7[0][0]          │
└───────────────────────────┴────────────────────────┴────────────────┴────────────────────────┘

 Total params: 8,471 (33.09 KB)

 Trainable params: 8,471 (33.09 KB)

 Non-trainable params: 0 (0.00 B)

In [19]:
model.save("my_model.h5")

In [20]:
# get batch of 128 for overfit model
X_trainOverfitBatch = X_train[:128]
Y_trainOverfitBatch = Y_train[:128]


In [21]:
networkHistory = model.fit(X_trainOverfitBatch, Y_trainOverfitBatch, epochs=800, batch_size=128, verbose = 1)

Epoch 1/800
1/1 ━━━━━━━━━━━━━━━━━━━━ 9s 9s/step - accuracy: 0.0781 - loss: 1.9989
Epoch 2/800
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 109ms/step - accuracy: 0.1406 - loss: 1.9694
Epoch 3/800
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 102ms/step - accuracy: 0.2109 - loss: 1.9404
Epoch 4/800
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 232ms/step - accuracy: 0.2812 - loss: 1.9112
Epoch 5/800
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 146ms/step - accuracy: 0.3047 - loss: 1.8817
Epoch 6/800
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 111ms/step - accuracy: 0.3516 - loss: 1.8504
Epoch 7/800
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 117ms/step - accuracy: 0.3906 - loss: 1.8168
Epoch 8/800
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 86ms/step - accuracy: 0.4453 - loss: 1.7809
Epoch 9/800
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 73ms/step - accuracy: 0.4609 - loss: 1.7442
Epoch 10/800
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 78ms/step - accuracy: 0.4844 - loss: 1.7083
Epoch 11/800
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 210ms/step - accuracy: 0.4688 - loss: 1.6732
Epoch 12/800
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 226ms/step - accuracy: 0.476

In [22]:
# evaluate on the validation set
val_loss, val_acc = model.evaluate(X_validation, Y_validation, verbose=1)

3632/3632 ━━━━━━━━━━━━━━━━━━━━ 8s 2ms/step - accuracy: 0.5666 - loss: 6.8291


In [23]:
num_params = model.count_params()

# final results
final_train_loss = networkHistory.history['loss'][-1]

# print the results
print(f"Number of parameters: {num_params}")
print(f"Final training loss: {final_train_loss}")
print(f"Final validation loss: {val_loss}")

Number of parameters: 8471
Final training loss: 0.0007975957123562694
Final validation loss: 6.827878475189209
